Machine Learning and Natural Language Processing


In [170]:
import pandas as pd
import os
os.chdir('C:/Users/Swati Arora/Data Science using Python')

In [171]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [172]:
train.head()

,username,gender,status,description
0,vimal20011,M,"{u'payment_verified': False, u'identity_verifi...",A team of 5 working on various projects relate...
1,sheom,M,"{u'payment_verified': True, u'identity_verifie...",We are an IT solution and service provider com...
2,ezbik,M,"{u'payment_verified': False, u'identity_verifi...",System administration is my work & hobby.
3,angelme,F,"{u'payment_verified': False, u'identity_verifi...",Good day! Thank you for taking some time to ch...
4,snitch1,M,"{u'payment_verified': False, u'identity_verifi...",I build good relation with clients and deliver...


In [173]:
test.head()

,username,status,description
0,nazrulmadina,"{u'payment_verified': False, u'identity_verifi...",I am one of Self-employed person having more t...
1,sehrishwarraich,"{u'payment_verified': False, u'identity_verifi...",i am sehrish warraich.I do my job sincerelly a...
2,samadhinie,"{u'payment_verified': False, u'identity_verifi...","Since 2006, Web based solution provider (Web e..."
3,ebottabi,"{u'payment_verified': False, u'identity_verifi...",Founder of a geolocation service developed on...
4,mrjimoy,"{u'payment_verified': False, u'identity_verifi...","Me? I am Jimmy, meaning honest, courageous, or..."


## 1. Predicting Gender with Username
Some potential features of usernames: whether it has capital letters, whether it has digits, number of characters, number of vowels, first and last letters, etc. See http://www.nltk.org/book/ch06.html for some related code.

In [174]:
import nltk
import random

In [175]:
def extract_features(name):
    vowels = "aAeEiIoOuU"
    return {
        'last_char': name[-1],
        'last_two': name[-2:],
        #'last_three':name[-3:],
        'first': name[0],
        'first2': name[:1],
        #'first3':name[:2],
        'upper':any(x.isupper() for x in name),
        'digit':any(x.isdigit() for x in name),
        'length':sum(c.isalpha() for c in name),
        'vowel': sum(name.count(c) for c in vowels)
       
    }

# commented first 3 and last three features since it was decreasing the AUC and accuracy of the model

In [176]:
username =train.username
gender = train.gender

In [177]:
feature_sets =[(extract_features(n),g) for n, g in zip(username,gender)]

In [178]:
# this function calculate the True Negative , True Positive, False Negative and False Positive values for model
true_pos=true_neg=false_pos=false_neg=0
def sens_spec(gen,pgen):
    tp=tn=fp=fn=0
    for g,pg in zip(gen, pgen):
        if g==pg=='M':
            tp = tp+1
        elif g==pg=='F':
            tn = tn+1
        elif g=='M' and pg=='F':
            fn=fn+1
        elif g=='F' and pg=='M':
            fp=fp+1
    return(tp,tn,fp,fn)

# Decision Tree Classifier

In [179]:
from sklearn.model_selection import KFold
import numpy as np
k_fold = KFold(n_splits=5, shuffle=True)
accu = []
for train_idx, test_idx in k_fold.split(feature_sets):
    training = [feature_sets[i] for i in train_idx]
    testing = [feature_sets[i] for i in test_idx]
    classifier_dt = nltk.classify.DecisionTreeClassifier.train(feature_sets, entropy_cutoff=0,support_cutoff=0)   
    accu.append( nltk.classify.util.accuracy(classifier_dt, testing) )
    print('accuracy:', accu[len(accu)-1])    
print('CV mean accuracy:', np.mean(accu))  

accuracy: 0.9848
accuracy: 0.9768
accuracy: 0.9824
accuracy: 0.9816
accuracy: 0.9831865492393915
CV mean accuracy: 0.981757309848


In [180]:
# Predicting the gender suinf model
name = ([extract_features(n) for n in username])
pred_gender_train = [classifier_dt.classify(n) for n in name]

In [181]:
# Calling the function and calcultaing Sensitivity and Specificity for AUC calculation
true_pos,true_neg,false_pos,false_neg = sens_spec(gender,pred_gender_train)
TPR = true_pos/(true_pos+false_neg)
TNR = true_neg/( true_neg+false_pos)

In [182]:
#AUC
roc_auc =TPR *(1-TNR)
print(roc_auc)

0.08617753244916244


# Naive Bayes Classifier for Username

In [183]:
from sklearn.model_selection import KFold
import numpy as np
k_fold = KFold(n_splits=5, shuffle=True)
accu = []
for train_idx, test_idx in k_fold.split(feature_sets):
    training = [feature_sets[i] for i in train_idx]
    testing = [feature_sets[i] for i in test_idx]
    classifier_nb = nltk.NaiveBayesClassifier.train(training)   
    accu.append( nltk.classify.util.accuracy(classifier_nb, testing) )
    print('accuracy:', accu[len(accu)-1])    
print('CV mean accuracy:', np.mean(accu))  

accuracy: 0.78
accuracy: 0.7792
accuracy: 0.776
accuracy: 0.7888
accuracy: 0.7918334667734187
CV mean accuracy: 0.783166693355


In [184]:
# Most informative features
print (classifier_nb.show_most_informative_features(20))

Most Informative Features
                last_two = 'va'                F : M      =     10.4 : 1.0
                last_two = 'nn'                F : M      =      7.6 : 1.0
                last_two = '3d'                F : M      =      5.8 : 1.0
                last_two = 'm1'                F : M      =      5.8 : 1.0
                last_two = 'm3'                F : M      =      5.8 : 1.0
                last_two = 'a6'                F : M      =      5.8 : 1.0
                last_two = 'iv'                F : M      =      5.8 : 1.0
                last_two = '30'                F : M      =      5.8 : 1.0
                last_two = 'ju'                F : M      =      5.8 : 1.0
                last_two = '47'                F : M      =      5.5 : 1.0
                last_two = 'jo'                F : M      =      4.9 : 1.0
                last_two = 'ek'                F : M      =      4.9 : 1.0
                last_two = 'bd'                M : F      =      4.5 : 1.0

In [185]:
username_test = test.username


In [186]:
# Predicting the gender suinf model
name = ([extract_features(n) for n in username])
pred_gender_train = [classifier_nb.classify(n) for n in name]

In [187]:
# Calling the function and calcultaing Sensitivity and Specificity for AUC calculation
true_pos,true_neg,false_pos,false_neg = sens_spec(gender,pred_gender_train)
TPR = true_pos/(true_pos+false_neg)
TNR = true_neg/( true_neg+false_pos)

In [188]:
#AUC
roc_auc =TPR *(1-TNR)
print(roc_auc)

0.7802253760196143


In [189]:
test_set = ([extract_features(n) for n in username_test])
#zz = classifier.classify([extract_features(n) for n in username_test]) 

In [190]:
pred_gender = [classifier_nb.classify(n) for n in test_set]
pred_gender[:5]

['F', 'M', 'M', 'M', 'M']

In [191]:
# Storing the test predictions in a list named pred_gender1
zz = pd.DataFrame({'prediction':pred_gender,'username':username_test })
zz.to_csv('pred_gender1.csv', index=False)

# Maximum Entropy Classifier

In [192]:
from sklearn.model_selection import KFold
import numpy as np
k_fold = KFold(n_splits=5, shuffle=True)
accu = []
for train_idx, test_idx in k_fold.split(feature_sets):
    train = [feature_sets[i] for i in train_idx]
    test = [feature_sets[i] for i in test_idx]
    classifier2_me = nltk.classify.MaxentClassifier.train(train, trace=3, max_iter=5)   
    accu.append( nltk.classify.util.accuracy(classifier2_me, testing) )
    print('accuracy:', accu[len(accu)-1])    
print('CV mean accuracy:', np.mean(accu)) 

  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.815
             2          -0.35212        0.815
             3          -0.34650        0.815
             4          -0.34146        0.815
         Final          -0.33703        0.815
accuracy: 0.811048839071257
  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.813
             2          -0.35567        0.813
             3          -0.34922        0.813
             4          -0.34347        0.813
         Final          -0.33846        0.814
accuracy: 0.8118494795836669
  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.812
             2          -0.35764        0.812
             3        

In [193]:
pred_gender_train = [classifier2_me.classify(n) for n in name]

In [194]:
# Calling the function and calcultaing Sensitivity and Specificity for AUC calculation
true_pos,true_neg,false_pos,false_neg = sens_spec(gender,pred_gender_train)
TPR = true_pos/(true_pos+false_neg)
TNR = true_neg/( true_neg+false_pos)

In [195]:
#AUC
roc_auc =TPR *(1-TNR)
print(roc_auc)

0.9965782720273738


In [196]:
pred_gender = [classifier2_me.classify(n) for n in test_set]
pred_gender[:5]

['M', 'M', 'M', 'M', 'M']

In [197]:
# Storing the test predictions in a list named pred_gender1
zz = pd.DataFrame({'prediction':pred_gender,'username':username_test })
zz.to_csv('pred_gender4.csv', index=False)

## 2. Predicting Gender with Description
The updated notebook for lecture 11 might be of some help, which now includes demo code for making predictions with NLTK classifier.

In [198]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
ps = PorterStemmer()
from nltk.tokenize import word_tokenize
def preprocess(text):
    return [ps.stem(w) for w in word_tokenize(text.lower()) 
             if w not in string.punctuation and w not in stopwords.words('english')] 

In [199]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [200]:
test.head()
description = train.description
description_test = test.description

In [201]:
des_words =[(preprocess(des),g) for des,g in zip(description,gender)]
#des_words[1]

In [202]:
len(description_test)

2679

In [203]:
des_words_test =[preprocess(des) for des in description_test]
#des_words_test[1]

In [204]:
# flatten the words into one list
all_words = [w for words, c in des_words for w in words]
all_words_test = [w for words in des_words_test for w in words]

In [205]:
#all_words_test[:20]

In [206]:
# Calculating word frequency
words_freq = nltk.FreqDist(all_words)
words_freq_test = nltk.FreqDist(all_words_test)

In [207]:
selected_words = [word for word, freq in words_freq.items() if freq>1]
print('train data: Before:',len(words_freq), ', after:', len(selected_words))
selected_words_test = [word for word, freq in words_freq_test.items() if freq>1]
print('Test data: Before:',len(words_freq_test), ', after:', len(selected_words_test))


train data: Before: 36128 , after: 9953
Test data: Before: 20142 , after: 5820


In [208]:
des_words = [([w for w in words if w in selected_words], c) for words, c in des_words]
des_words_test = [[w for w in words if w in selected_words] for words in des_words_test]

In [209]:
#des_words[:2]

In [210]:
def extract_features(words):
    ''' simply using words counts'''
    return nltk.FreqDist(words)

In [211]:
feat = [(extract_features(w),g) for w,g in des_words]
feat_test = [extract_features(w) for w in des_words_test]

# Naive Bayes Classifier

In [212]:

from sklearn.model_selection import KFold
import numpy as np
k_fold = KFold(n_splits=5, shuffle=True)
accu = []
for train_idx, test_idx in k_fold.split(feat):
    training = [feat[i] for i in train_idx]
    testing = [feat[i] for i in test_idx]
    classifier_nb = nltk.NaiveBayesClassifier.train(training)   
    accu.append( nltk.classify.util.accuracy(classifier_nb, testing) )
    print('accuracy:', accu[len(accu)-1])    
print('CV mean accuracy:', np.mean(accu))    

accuracy: 0.5744
accuracy: 0.5432
accuracy: 0.5744
accuracy: 0.5752
accuracy: 0.5508406725380304
CV mean accuracy: 0.563608134508


In [213]:
z =[preprocess(des) for des in description]
z = [([w for w in words if w in selected_words]) for words in z]
feat_train = [extract_features(w) for w in z]

In [214]:
pred_gender_train = [classifier_nb.classify(n) for n in feat_train]

In [215]:
true_pos,true_neg,false_pos,false_neg = sens_spec(gender,pred_gender_train)
TPR = true_pos/(true_pos+false_neg)
TNR = true_neg/( true_neg+false_pos)
FPR = false_pos/(false_pos+true_neg) 

In [216]:
#AUC
roc_auc =TPR *(1-TNR)
print(roc_auc)

0.07182934468520777


In [261]:
print (classifier_nb.show_most_informative_features(5))

Most Informative Features
                   femal = 1                   F : M      =     21.5 : 1.0
                     mom = 1                   F : M      =     18.0 : 1.0
                lifestyl = 1                   F : M      =     15.7 : 1.0
                 inquiri = 1                   F : M      =     15.7 : 1.0
                    take = 3                   F : M      =     15.7 : 1.0
None


# Maximum Entropy Classifier

In [218]:
k_fold = KFold(n_splits=5, shuffle=True)
accu = []
for train_idx, test_idx in k_fold.split(feat):
    train = [feat[i] for i in train_idx]
    test = [feat[i] for i in test_idx]
    classifier2_me = nltk.classify.MaxentClassifier.train(train, trace=3, max_iter=5)       
    accu.append( nltk.classify.util.accuracy(classifier2_me, test) )
    print('accuracy:', accu[len(accu)-1])    
print('CV mean accuracy:', np.mean(accu)) 

  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.815
             2          -0.40123        0.803
             3          -0.34830        0.789
             4          -0.33270        0.774
         Final          -0.32500        0.768
accuracy: 0.7312
  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.810
             2          -0.41520        0.789
             3          -0.36496        0.771
             4          -0.35007        0.754
         Final          -0.34343        0.750
accuracy: 0.748
  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.814
             2          -0.35900        0.803
             3          -0.31551        0.794


In [219]:
pred_gender_train = [classifier2_me.classify(n) for n in feat_train]

In [220]:
#classifier = nltk.classifier2_me.MaxentClassifier.train(feat, trace=3, max_iter=5)

In [221]:
true_pos,true_neg,false_pos,false_neg = sens_spec(gender,pred_gender_train)
TPR = true_pos/(true_pos+false_neg)
TNR = true_neg/( true_neg+false_pos)
FPR = false_pos/(false_pos+true_neg) 

In [222]:
#AUC
roc_auc =TPR *(1-TNR)
print(roc_auc)

0.9153240201262268


In [223]:
pred_test = [classifier2_me.classify(n) for n in feat_test]

In [224]:
#nltk.classify.util.accuracy(classifier, feat_test)
# support your predictions are stored in a list named pred_gender1
zz = pd.DataFrame({'prediction':pred_test,'Username':username_test })
zz.to_csv('pred_gender3.csv', index=False)

# Decision Tree Classifier

In [225]:
from sklearn.model_selection import KFold
import numpy as np
k_fold = KFold(n_splits=5, shuffle=True)
accu = []
for train_idx, test_idx in k_fold.split(feat):
    training = [feature_sets[i] for i in train_idx]
    testing = [feature_sets[i] for i in test_idx]
    classifier_dt = nltk.classify.DecisionTreeClassifier.train(feature_sets, entropy_cutoff=0,support_cutoff=0)   
    accu.append( nltk.classify.util.accuracy(classifier_dt, testing) )
    print('accuracy:', accu[len(accu)-1])    
print('CV mean accuracy:', np.mean(accu)) 

accuracy: 0.9784
accuracy: 0.9856
accuracy: 0.9784
accuracy: 0.9824
accuracy: 0.9839871897518014
CV mean accuracy: 0.98175743795


In [226]:
pred_gender_train = [classifier_dt.classify(n) for n in feat_train]

In [227]:
true_pos,true_neg,false_pos,false_neg = sens_spec(gender,pred_gender_train)
TPR = true_pos/(true_pos+false_neg)
TNR = true_neg/( true_neg+false_pos)
FPR = false_pos/(false_pos+true_neg) 

In [228]:
#AUC
roc_auc =TPR *(1-TNR)
print(roc_auc)

1.0


In [229]:
pred_test = [classifier_dt.classify(n) for n in feat_test]

In [230]:
#nltk.classify.util.accuracy(classifier, feat_test)
# support your predictions are stored in a list named pred_gender1
zz = pd.DataFrame({'prediction':pred_test,'Username':username_test })
zz.to_csv('pred_gender2.csv', index=False)

## 3. Predicting Gender with Username, Description, and Status
If you need to merge multiple dict-format features into one, check the following question: https://stackoverflow.com/questions/38987/how-to-merge-two-dictionaries-in-a-single-expression

In [231]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,username,gender,status,description
0,vimal20011,M,"{u'payment_verified': False, u'identity_verifi...",A team of 5 working on various projects relate...
1,sheom,M,"{u'payment_verified': True, u'identity_verifie...",We are an IT solution and service provider com...
2,ezbik,M,"{u'payment_verified': False, u'identity_verifi...",System administration is my work & hobby.
3,angelme,F,"{u'payment_verified': False, u'identity_verifi...",Good day! Thank you for taking some time to ch...
4,snitch1,M,"{u'payment_verified': False, u'identity_verifi...",I build good relation with clients and deliver...


In [232]:
# Parse Json format status as dictionary
from ast import literal_eval
status = train['status'].apply(literal_eval)
status_test = test['status'].apply(literal_eval)

# spliting the dictionary format status as multiple columns

In [233]:
# for train data
status_new = pd.DataFrame()
for l in status.iteritems():
    row = (l[1])
    df =pd.DataFrame(list(row.items()))
    df['row']=l[0]
    #print(df)
    status_new = status_new.append(df.pivot(index='row',columns=0, values=1))
    #print(status_new)
    

In [234]:
# for test data
status_test_new = pd.DataFrame()
for l in status_test.iteritems():
    row = (l[1])
    df =pd.DataFrame(list(row.items()))
    df['row']=l[0]
    #print(df)
    status_test_new = status_test_new.append(df.pivot(index='row',columns=0, values=1))
    #print(status_new)

In [235]:
#df = pd.DataFrame(list(a_dictionary.items()), columns = ['column1', 'column2'])
status_new.head()

,deposit_made,email_verified,facebook_connected,identity_verified,payment_verified,phone_verified,profile_complete
row,,,,,,,
0,True,True,False,False,False,False,True
1,True,True,True,False,True,False,True
2,False,True,False,False,False,False,True
3,True,True,False,False,False,True,True
4,False,True,False,False,False,False,True


In [236]:
# description
feat_test[1]

FreqDist({'better': 2,
          'data': 1,
          'entri': 1,
          'experienc': 1,
          'job': 2,
          'mission': 1,
          'next': 1,
          'previou': 1,
          'submit': 1,
          'time': 1})

In [237]:
# Username
feature_sets[1][0]

{'digit': False,
 'first': 's',
 'first2': 's',
 'last_char': 'm',
 'last_two': 'om',
 'length': 5,
 'upper': False,
 'vowel': 2}

In [238]:
# Converting the status to dictionary

In [239]:
st = status_new.to_dict(orient='records')
st_test = status_test_new.to_dict(orient='records')

In [240]:
st[1]

{'deposit_made': True,
 'email_verified': True,
 'facebook_connected': True,
 'identity_verified': False,
 'payment_verified': True,
 'phone_verified': False,
 'profile_complete': True}

# Merge Username, Description and Status dictionary in single expression using zip and unzip

In [241]:
# for train data
comb_features = []
for user,des,sta in zip(feature_sets,feat,st):
    #print(user)
    #print(des)
    z = {**user[0], **des[0]}
    x = {**z,**sta}
    comb_features.append(x)
    

In [242]:
# for test data
comb_features_test = []
for user,des,sta in zip(test_set,feat_test,st_test):
    #print(user)
    #print(des)
    z = {**user, **des}
    x = {**z,**sta}
    comb_features_test.append(x)

In [243]:
len(comb_features)
#len(feature_sets)
#len(feat)
#comb_features[0]

6249

In [244]:
comb_features_train = [(w,g) for w,g in zip(comb_features,gender)]

In [245]:
comb_features_train_auc = [w for w in comb_features]
comb_features_test = [w for w in comb_features_test]

# Naive Bayes Classifier

In [246]:
from sklearn.model_selection import KFold
import numpy as np
k_fold = KFold(n_splits=5, shuffle=True)
accu = []
for train_idx, test_idx in k_fold.split(comb_features_train):
    training = [comb_features_train[i] for i in train_idx]
    testing = [comb_features_train[i] for i in test_idx]
    classifier_nb = nltk.NaiveBayesClassifier.train(training)   
    accu.append( nltk.classify.util.accuracy(classifier_nb, testing) )
    print('accuracy:', accu[len(accu)-1])    
print('CV mean accuracy:', np.mean(accu))  

accuracy: 0.5776
accuracy: 0.5528
accuracy: 0.588
accuracy: 0.588
accuracy: 0.5876701361088871
CV mean accuracy: 0.578814027222


In [247]:
pred_gender_train = [classifier_nb.classify(n) for n in comb_features_train_auc]

In [248]:
true_pos,true_neg,false_pos,false_neg = sens_spec(gender,pred_gender_train)
TPR = true_pos/(true_pos+false_neg)
TNR = true_neg/( true_neg+false_pos)
FPR = false_pos/(false_pos+true_neg) 


In [249]:
#AUC
roc_auc =TPR *(1-TNR)
print(roc_auc)

0.06387382714885191


In [250]:
print (classifier_nb.show_most_informative_features(20))

Most Informative Features
                   femal = 1                   F : M      =     21.5 : 1.0
                     mom = 1                   F : M      =     18.0 : 1.0
                lifestyl = 1                   F : M      =     15.7 : 1.0
                 inquiri = 1                   F : M      =     15.7 : 1.0
                    take = 3                   F : M      =     15.7 : 1.0
                   woman = 1                   F : M      =     14.6 : 1.0
                   encod = 1                   F : M      =     14.6 : 1.0
                last_two = 'va'                F : M      =     13.1 : 1.0
                  well.i = 1                   F : M      =     12.9 : 1.0
                accommod = 1                   F : M      =     12.9 : 1.0
                     cat = 1                   F : M      =     12.9 : 1.0
                  museum = 1                   F : M      =     12.9 : 1.0
            well-written = 1                   F : M      =     12.9 : 1.0

# Maximum Entropy Classifier

In [251]:

accu = []
for train_idx, test_idx in k_fold.split(comb_features_train):
    train = [comb_features_train[i] for i in train_idx]
    test = [comb_features_train[i] for i in test_idx]
    classifier2_me = nltk.classify.MaxentClassifier.train(train, trace=3, max_iter=5)       
    accu.append( nltk.classify.util.accuracy(classifier2_me, test) )
    print('accuracy:', accu[len(accu)-1])    
print('CV mean accuracy:', np.mean(accu)) 

  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.812
             2          -0.28269        0.814
             3          -0.23671        0.814
             4          -0.22565        0.813
         Final          -0.22114        0.812
accuracy: 0.8144
  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.814
             2          -0.33672        0.815
             3          -0.28288        0.810
             4          -0.26800        0.800
         Final          -0.26143        0.795
accuracy: 0.7896
  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.809
             2          -0.28255        0.810
             3          -0.23737        0.810

In [252]:
pred_gender_train = [classifier2_me.classify(n) for n in comb_features_train_auc]

In [253]:
true_pos,true_neg,false_pos,false_neg = sens_spec(gender,pred_gender_train)
TPR = true_pos/(true_pos+false_neg)
TNR = true_neg/( true_neg+false_pos)
FPR = false_pos/(false_pos+true_neg) 

In [254]:
#AUC
roc_auc =TPR *(1-TNR)
print(roc_auc)

0.9899570936866425


In [255]:
pred_test = [classifier2_me.classify(n) for n in comb_features_test]

In [256]:
# support your predictions are stored in a list named pred_gender5
zz = pd.DataFrame({'prediction':pred_test,'Username':username_test })
zz.to_csv('pred_gender5.csv', index=False)

# Decision Tree Classifier

In [257]:
from sklearn.model_selection import KFold
import numpy as np
k_fold = KFold(n_splits=5, shuffle=True)
accu = []
for train_idx, test_idx in k_fold.split(comb_features_train):
    training = [feature_sets[i] for i in train_idx]
    testing = [feature_sets[i] for i in test_idx]
    classifier_dt = nltk.classify.DecisionTreeClassifier.train(feature_sets, entropy_cutoff=0,support_cutoff=0)   
    accu.append( nltk.classify.util.accuracy(classifier_dt, testing) )
    print('accuracy:', accu[len(accu)-1])    
print('CV mean accuracy:', np.mean(accu)) 

accuracy: 0.98
accuracy: 0.9848
accuracy: 0.9824
accuracy: 0.9824
accuracy: 0.9791833466773419
CV mean accuracy: 0.981756669335


In [258]:
pred_gender_train = [classifier_dt.classify(n) for n in comb_features_train_auc]

In [259]:
true_pos,true_neg,false_pos,false_neg = sens_spec(gender,pred_gender_train)
TPR = true_pos/(true_pos+false_neg)
TNR = true_neg/( true_neg+false_pos)
FPR = false_pos/(false_pos+true_neg) 

In [260]:
#AUC
roc_auc =TPR *(1-TNR)
print(roc_auc)

0.10816937553464505


# Summary

For Username: I have extracted the features giving the best model while removing ones which
reduces the model performance. The result of three models are as follows:
<br><br>
- Decision Tree Clasifier
-- Accuracy: 0.98 / AUC : 0.086 
- Naive Bayes Classifier
--Accuracy: 0.78 / AUC : 0.785
- Maximum Entropy Classifier
--Accuracy: 0.82 / AUC: 0.99

<br><br>
For Description: stemming the sentence into words and removed the stopwords and punctuations
Created the disctionary by calculating the word frequency and then implementing the models:

- Naive Bayes Clasifier
-- Accuracy: 0.56 / AUC : 0.08
- Maximum Entropy Clasifier
-- Accuracy: 0.76 / AUC : 0.81
- Decision Tree Clasifier
-- Accuracy: 0.98 / AUC : 1.0

<br><br>
For predicting Gender using username, description and Status: I have used same format for username and description as above.
I splitted the status as multiple column of value True and False, and then converted it to dictionary format.
Merged all the features as single unit and applied the models to predict Gender:

- Naive Bayes Classifier
--Accuracy: 0.58 / AUC : 0.05
- Maximum Entropy Clasifier
-- Accuracy: 0.8 / AUC : 0.86
- Decision Tree Clasifier
-- Accuracy: 0.98 / AUC : 0.1
<br><br>
For few models even if the accuracy is more the AUC is less. SO, I have evaluated the model based on both Accuracy and AUC values. The best 5 model according to me are saved.
